In [83]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt

from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [84]:
df = pd.read_csv("partial_database.csv")

# Read class labels
f = open("labelsDefault.txt", "r")
labels=f.read().splitlines()
labels.pop(0)

# Add y column, which is the target variable
df['y'] = labels
print(df["y"].unique())

# Get dataset only for class 1
df=df.loc[df['y'] == "01"]
print(df["y"].unique())

['01' '02' '03' '04' '05' '06' '07' '08' '09' '10' '11' '12' '13' '14'
 '15' '16' '17' '18' '19' '20' '21' '22']
['01']


In [85]:
target = torch.tensor(df['y'].values.astype(np.int_))
data = torch.tensor(df.drop('y', axis = 1).values.astype(np.float32)) 
df_tensor = torch.utils.data.TensorDataset(data, target)

In [86]:
#Load data into a PyTorch dataset
train_loader = DataLoader(dataset=df_tensor, batch_size=32, shuffle=True)
train_features, train_labels = next(iter(train_loader))

print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

Feature batch shape: torch.Size([32, 17])
Labels batch shape: torch.Size([32])


In [87]:
# Creating a PyTorch class
# 18 ==> 3 ==> 18

class AE(torch.nn.Module):
	def __init__(self):
		super().__init__()
		
		self.encoder = torch.nn.Sequential(
			torch.nn.Linear(17, 6),
			torch.nn.ReLU(),
			torch.nn.Linear(6, 3),
    		torch.nn.ReLU()
		)
		
		self.decoder = torch.nn.Sequential(
			torch.nn.Linear(3, 6),
			torch.nn.ReLU(),
			torch.nn.Linear(6, 17),
			torch.nn.ReLU()
		)

	def forward(self, x):
		encoded = self.encoder(x)
		decoded = self.decoder(encoded)
		return decoded


In [88]:
# Model Initialization
model = AE()

# Validation using MSE Loss function
loss_function = torch.nn.MSELoss()

# Using an Adam Optimizer with lr = 0.1
optimizer = torch.optim.Adam(model.parameters(),
							lr = 1e-1,
							weight_decay = 1e-8)


In [89]:
epochs = 100
losses = []
writer = SummaryWriter()

for epoch in range(epochs):
    for data in train_loader:
        features, _ = data
        optimizer.zero_grad()
        reconstructed = model(features)
        loss = loss_function(reconstructed, features)
        loss.backward()
        #reconstruction_error = torch.mean((reconstructed - features)**2)
        optimizer.step()
        losses.append(loss.item())

    print(f'Epoch: {epoch}, Loss: {loss.item():.4f}')
    #print(f'Epoch: {epoch}, Reconstruction error: {reconstruction_error.item():.4f}')
    writer.add_scalar('Loss', loss.item(), epoch)
    #writer.add_scalar('Error', reconstruction_error.item(), epoch)



Epoch: 0, Loss: 0.2229
Epoch: 1, Loss: 0.1460
Epoch: 2, Loss: 0.1950
Epoch: 3, Loss: 0.2154
Epoch: 4, Loss: 0.0960
Epoch: 5, Loss: 0.1814
Epoch: 6, Loss: 0.0760


Epoch: 7, Loss: 0.1981
Epoch: 8, Loss: 0.0887
Epoch: 9, Loss: 0.0631
Epoch: 10, Loss: 0.1419
Epoch: 11, Loss: 0.1023
Epoch: 12, Loss: 0.0793
Epoch: 13, Loss: 0.0789
Epoch: 14, Loss: 0.0986
Epoch: 15, Loss: 0.1815
Epoch: 16, Loss: 0.0847
Epoch: 17, Loss: 0.2143
Epoch: 18, Loss: 0.1038
Epoch: 19, Loss: 0.0731
Epoch: 20, Loss: 0.0648
Epoch: 21, Loss: 0.0766
Epoch: 22, Loss: 0.0810
Epoch: 23, Loss: 0.0888
Epoch: 24, Loss: 0.0931
Epoch: 25, Loss: 0.0874
Epoch: 26, Loss: 0.1093
Epoch: 27, Loss: 0.0480
Epoch: 28, Loss: 0.0830
Epoch: 29, Loss: 0.1000
Epoch: 30, Loss: 0.0708
Epoch: 31, Loss: 0.0893
Epoch: 32, Loss: 0.0798
Epoch: 33, Loss: 0.2202
Epoch: 34, Loss: 0.0930
Epoch: 35, Loss: 0.0978
Epoch: 36, Loss: 0.1048
Epoch: 37, Loss: 0.0811
Epoch: 38, Loss: 0.1653
Epoch: 39, Loss: 0.0648
Epoch: 40, Loss: 0.0831
Epoch: 41, Loss: 0.1273
Epoch: 42, Loss: 0.2252
Epoch: 43, Loss: 0.0785
Epoch: 44, Loss: 0.0782
Epoch: 45, Loss: 0.0849
Epoch: 46, Loss: 0.1203
Epoch: 47, Loss: 0.1299
Epoch: 48, Loss: 0.

In [90]:
#Load tensorboard
#tensorboard --logdir=runs
#http://localhost:6006/